# PS-04: k-NN & TF-IDF

Partners:
Katie Goulding & Aneesha Nanda


# Part 1: The Data

In [765]:
import pandas as pd 
import numpy as np
from numpy.linalg import norm
import collections  
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier


In [766]:
texts = pd.read_csv('texts.csv', delimiter= '\t', quoting=2)

In [767]:
texts.head()

,name,size,lines,chunkid,chunk
0,balbulus-early-life-charlemagne,259062.0,4394.0,1.0,\nTitle: Early Lives of Charlemagne by Eginhar...
1,balbulus-early-life-charlemagne,259062.0,4394.0,2.0,"\n\nThe notes, keyed to line numbers in the so..."
2,balbulus-early-life-charlemagne,259062.0,4394.0,3.0,\n From a bronze statuette in the Musé...
3,balbulus-early-life-charlemagne,259062.0,4394.0,4.0,\n _A lui finit la dissolution ...
4,balbulus-early-life-charlemagne,259062.0,4394.0,5.0,public opinion in regard to the meaning of fal...


In [768]:
names = df.groupby("name")
print("Number of texts: %i" % texts.shape[0])
print("Number of titles: %i" % len(np.unique(texts.name.astype('str'))))
print(names["name"].nunique())

Number of texts: 12924
Number of titles: 29
name
balbulus-early-life-charlemagne                 1
beesly-queen-elizabeth                          1
bible                                           1
carroll-alice-wonderland                        1
chipman-earliest-electromagnetic-instruments    1
cia-world-factbook-1992                         1
eckstein-quintus-claudius                       1
fisher-quaker-colonies                          1
gallienne-quest-of-golden-girl                  1
gordon-quiet-talks-crowned-christ               1
hardy-madding-crowd                             1
infiltrating-open-systems                       1
kant-metaphysical-elements-ethics               1
karn-snowflakes                                 1
milton-paradise-lost                            1
naval-academy-sound-military-decision           1
newsgroup                                       1
paper-compact-hash-tables                       1
paper-data-compression                          1
p

# 1.1 Bag of Words

In [769]:
from sklearn.cross_validation import train_test_split
train_size = 1000
test_size = 10
train, test = train_test_split(texts, train_size = train_size, test_size = test_size, random_state = 5)

In [770]:
x_test= test.chunk.values.astype("U")
y_test = test.name.values.astype("U")
x_train = train.chunk.values.astype("U")
y_train = train.name.values.astype("U")

In [771]:
sentences = np.concatenate((x_train, x_test),axis=0)

In [772]:
vectorizer = CountVectorizer(min_df=0)
vectorizer.fit(sentences)
X = vectorizer.transform(sentences).toarray()
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [773]:
# Cosine Similarity
def similarity(x, y):
    dot_product = np.dot(x, y)
    norm_x = np.linalg.norm(x)
    norm_y = np.linalg.norm(y)
    return dot_product / (norm_x * norm_y)

# Verify similarities
print(similarity(X[1], X[1]))
print(similarity(X[1], X[2]))
print(similarity(X[1], X[8]))
print(similarity(X[5], X[3]))
print(similarity(X[4], X[2]))

1.0000000000000002
0.5364020879611584
0.5039494801360922
0.5209433025814969
0.6574704794308697


In [774]:
Xtrain = X[:train_size]
Xtest = X[-test_size:]
k = 1
def getNeighbors(train, test, k):
    preds = []
    for i in range(len(test)):
        similarities = []
        test_instance = test[i]
        print(y_test[i])
        for j in range(len(Xtrain)):
            cos_sim = similarity(test_instance, train[j])
            similarities.append(cos_sim)
        s = pd.DataFrame({'name': y_train, 's': similarities})
        s = s.sort_values('s', ascending = False)    
        freq = collections.Counter(s.head(k).name.values)
        preds.append(list(freq.keys())[0])
    return np.mean(y_test == preds)

getNeighbors(Xtrain, Xtest, k)
   

vaneeden-quest
bible
cia-world-factbook-1992
bible
why-speech-output
bible
newsgroup
newsgroup
why-speech-output
hardy-madding-crowd


0.6

In [775]:
#Compute TF-IDF
tf_train = np.log(1+Xtrain)
tf_test = np.log(1+Xtest)

tfidf_train = tf_train * np.log(len(X)/(1+(X>0).sum(axis=0)))
tfidf_test = tf_test * np.log(len(X)/(1+(X>0).sum(axis=0)))

getNeighbors(tf_train, tf_test, k)


vaneeden-quest
bible
cia-world-factbook-1992
bible
why-speech-output
bible
newsgroup
newsgroup
why-speech-output
hardy-madding-crowd


0.8

From this, we can see that TF-IDF is more accurate (~80%) in comparison to Bag of Words (~60%) and is faster to run as well. The value of k however, didn't seem to impact the amount of accuracy for both algorithms. The larger sample sizes often gave slightly higher accuracies than lower ones. 

# Part 2: Rotten Tomatoes

In [778]:
data = pd.read_csv('rotten-tomatoes.csv')
new_df = data[['fresh', 'quote']]
new_df=new_df.dropna()
new_df=new_df.drop_duplicates()
new_df

,fresh,quote
0,fresh,"So ingenious in concept, design and execution ..."
1,fresh,The year's most inventive comedy.
2,fresh,A winning animated feature that has something ...
3,fresh,The film sports a provocative and appealing st...
4,fresh,"An entertaining computer-generated, hyperreali..."
5,fresh,"As Lion King did before it, Toy Story revived ..."
6,fresh,The film will probably be more fully appreciat...
7,fresh,Children will enjoy a new take on the irresist...
8,fresh,Although its computer-generated imagery is imp...
9,fresh,The result is a visionary roller-coaster ride ...


> The original data had about 13,442 cases. After removing the null values and reducing the redundancy within the original dataset, there are now 12,838 cases.

In [748]:
rt_train_size = 100
rt_test_size = 30
rot_train, rot_test = train_test_split(new_df, train_size = rt_train_size, test_size = rt_test_size, random_state = 5)


In [749]:
rotten_x_test= rot_test.quote.values.astype("U")
rotten_y_test = rot_test.fresh.values.astype("U")
rotten_x_train = rot_train.quote.values.astype("U")
rotten_y_train = rot_train.fresh.values.astype("U")

In [750]:
rotten_sentences = np.concatenate((rotten_x_train, rotten_x_test),axis=0)

In [751]:
vectorizer = CountVectorizer(min_df=0)
vectorizer.fit(rotten_sentences)
words = vectorizer.transform(rotten_sentences).toarray()
words

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [759]:
X_rot_train = words[:rt_train_size]
X_rot_test = words[-rt_test_size:]
rt_k = 5
def getNeighbors_RT (rot_train, rot_test, k):
    rt_preds = []
    for i in range(len(rot_test)):
        rt_similarities = []
        rt_test_instance = rot_test[i]
        print(rotten_y_test[i])
        for j in range(len(X_rot_train)):
            rt_cos_sim = similarity(rt_test_instance, rot_train[j])
            rt_similarities.append(rt_cos_sim)
        s = pd.DataFrame({'fresh': rotten_y_train, 's': rt_similarities})
        s = s.sort_values('s', ascending = False)    
        freq = collections.Counter(s.head(rt_k).fresh.values)
        rt_preds.append(list(freq.keys())[0])

    return np.mean(rotten_y_test == rt_preds)

getNeighbors_RT(X_rot_train, X_rot_test, rt_k)



fresh
fresh
fresh
rotten
fresh
rotten
fresh
fresh
fresh
fresh
rotten
fresh
rotten
fresh
fresh
rotten
fresh
rotten
rotten
rotten
fresh
fresh
fresh
fresh
fresh
rotten
fresh
rotten
rotten
fresh


0.6

In [760]:
rt_tf_train = np.log(1+X_rot_train)
rt_tf_test = np.log(1+X_rot_test)

rot_tfidf_train = rt_tf_train * np.log(len(words)/(1+(words>0).sum(axis=0)))
rot_tfidf_test = rt_tf_test * np.log(len(words)/(1+(words>0).sum(axis=0)))

getNeighbors_RT(rt_tf_train, rt_tf_test, rt_k)

fresh
fresh
fresh
rotten
fresh
rotten
fresh
fresh
fresh
fresh
rotten
fresh
rotten
fresh
fresh
rotten
fresh
rotten
rotten
rotten
fresh
fresh
fresh
fresh
fresh
rotten
fresh
rotten
rotten
fresh


0.6333333333333333

# Conclusion: 

From performing the KNN and TF-IDF algorithms on the different pieces of text and the rotten tomato reviews, we can conclude that the text pieces were better to perform the algorithms on, specifically TF-IDF. The algorithms may have performed more poorly on the Rotten Tomatoes data versus the texts because some keywords in the quote could've been misinterpreted. To gauge more accurately whether a not a movie review would be "fresh", we would have to do sentiment analysis to distinguish between positive and negative feelings within text. Due to this, it is harder for the algorithm to predict what would be considered "fresh" or "rotten" based on what the review entails. Overall, KNN gave on average about 20% less accuracy scores than TF-IDF. 